In [1]:
!pip install ray

In [2]:
import ray
ray.init(num_cpus=8)

{'metrics_export_port': 58354,
 'node_id': 'd3b367cf917f69cec7d4d1604ff53ba96fdf5344401d7d92ca5a8e3c',
 'node_ip_address': '172.28.0.2',
 'object_store_address': '/tmp/ray/session_2021-11-28_21-12-15_914244_603/sockets/plasma_store',
 'raylet_ip_address': '172.28.0.2',
 'raylet_socket_name': '/tmp/ray/session_2021-11-28_21-12-15_914244_603/sockets/raylet',
 'redis_address': '172.28.0.2:6379',
 'session_dir': '/tmp/ray/session_2021-11-28_21-12-15_914244_603',
 'webui_url': None}

In [3]:
from json import load
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')

import pandas as pd

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
PATH ='/content/drive/MyDrive/BigData/dataset proyecto/'

In [6]:
with open(PATH + '/MMHS150K_GT.json', 'r') as fp:
  data = load(fp)

In [7]:
df = pd.DataFrame.from_dict(data, orient='index')
df = df[['tweet_text', 'labels_str', 'tweet_url']]

In [8]:
_stopwords = set(stopwords.words("english") + ['\'', '’'])

def limpiar(t):
  without_links = ' '.join([word for word in t.split(' ') if word[:4] != 'http']) # quitar los links
  tokens = word_tokenize(without_links) # tokenizar (dividir en tokens)
  tokens = map(lambda token: token.encode('ascii', 'ignore').decode('ascii'), tokens)
  tokens_without_sw = [word.lower() for word in tokens if word.isalpha() and not word in _stopwords]
  return ' '.join(tokens_without_sw)

@ray.remote
def ray_limpiar(t):
  return limpiar(t)

In [11]:
df['tweet_text'] = df['tweet_text'].map(ray_limpiar.remote)

AttributeError: ignored

In [13]:
df['tweet_text'] = df['tweet_text'].map(ray.get)

In [14]:
df.head()

,tweet_text,labels_str,tweet_url
1114679353714016256,friskdontmiss nigga,"[Religion, Racist, Homophobe]",https://twitter.com/user/status/11146793537140...
1063020048816660480,my horses retarded,"[OtherHate, OtherHate, OtherHate]",https://twitter.com/user/status/10630200488166...
1108927368075374593,nigga on ma momma youngboy be spitting real sh...,"[NotHate, NotHate, NotHate]",https://twitter.com/user/status/11089273680753...
1114558534635618305,rt xxsugvngxx i ran holy nigga today,"[Racist, NotHate, NotHate]",https://twitter.com/user/status/11145585346356...
1035252480215592966,everybody calling nigger,"[Racist, NotHate, Racist]",https://twitter.com/user/status/10352524802155...
